In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    """read data from data file"""
    with open(filename, "rb") as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1
        self._labels = np.hstack(all_labels)
        #print(self._data.shape)
        #print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
                    
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' %i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

#batch_data, batch_labels = train_data.next_batch(10)
#print(batch_data)
#print(batch_labels)

In [3]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

x = tf.compat.v1.placeholder(tf.float32, [None,3072])
y = tf.compat.v1.placeholder(tf.int64, [None])
x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm=[0,2,3,1])

conv1 = tf.compat.v1.layers.conv2d(x_image,
                         32, 
                         (3,3), 
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv1'
                        )
pooling1 = tf.compat.v1.layers.max_pooling2d(conv1, 
                                   (2, 2),
                                   (2, 2),
                                   name = 'pool1'
                                  )

conv2 = tf.compat.v1.layers.conv2d(pooling1,
                         32, 
                         (3,3), 
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv2'
                        )
pooling2 = tf.compat.v1.layers.max_pooling2d(conv2, 
                                   (2, 2),
                                   (2, 2),
                                   name = 'pool2'
                                  )

conv3 = tf.compat.v1.layers.conv2d(pooling2,
                         32, 
                         (3,3), 
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv3'
                        )
pooling3 = tf.compat.v1.layers.max_pooling2d(conv3, 
                                   (2, 2),
                                   (2, 2),
                                   name = 'pool3'
                                  )

flatten = tf.compat.v1.layers.flatten(pooling3)
y_ = tf.compat.v1.layers.dense(flatten, 10)

loss = tf.compat.v1.losses.sparse_softmax_cross_entropy(labels = y, logits= y_)

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.compat.v1.train.AdamOptimizer(1e-3).minimize(loss)

/var/folders/n9/chhy6rnj2m3bh5gqhm0yjn6w0000gn/T/ipykernel_20546/3857448755.py:9: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv1 = tf.compat.v1.layers.conv2d(x_image,
/opt/anaconda3/lib/python3.9/site-packages/keras/legacy_tf_layers/convolutional.py:563: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/var/folders/n9/chhy6rnj2m3bh5gqhm0yjn6w0000gn/T/ipykernel_20546/3857448755.py:16: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  pooling1 = tf.compat.v1.layers.max_pooling2d(conv1,
/opt/anaconda3/lib/python3.9/site-packages/keras/legacy_tf_layers/pooling.py:568: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  retur

In [4]:
init = tf.compat.v1.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100

with tf.compat.v1.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x:batch_data, y:batch_labels})
        if (i+1) %500 == 0:
            print ("[Train] Step: %d, loss: %4.5f, acc:%4.5f" %(i, loss_val, acc_val))
        if (i+1) %5000 ==0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
                test_acc=np.mean(all_test_acc_val)
                print('[Test] Step: %d, acc: %4.5f' %(i+1, test_acc))
            

2022-02-11 15:18:38.631473: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[Train] Step: 499, loss: 0.94662, acc:0.70000
[Train] Step: 999, loss: 1.12866, acc:0.65000
